# Testing Individual components of the FV HE scheme

In [20]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [21]:
poly_modulus = 8
bit_sizes= [10]
plain_modulus = 8
# poly_modulus = 256
# bit_sizes= [20]
# plain_modulus = 256
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [22]:
print(ctx.param.coeff_modulus)

[1009]


In [23]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1008, 0, 0, 1008, 0, 0, 1008, 0]]


In [24]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [25]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1, 1]     [0, 1]     [1, 0, 0, 1]


### Decodes back to Integer

In [26]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

3
2
9


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [27]:
encrypter = Encryptor(ctx, pk)

In [28]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [29]:
decrypter = Decryptor(ctx, sk)

In [30]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [31]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

3     2     9


## Evaluator

In [32]:
eval = Evaluator(ctx)

In [33]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

5


In [34]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

5


In [35]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

5


### Verify result

In [36]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [37]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)

ciphertexts
[[64576, 67603, 968640, 123098, 988820, 611454, 851596, 65585]]
[[179602, 612463, 549905, 243169, 881866, 326916, 1001937, 302700]]
result
[[[0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0]]]


ct1 : [[[64, 67, 960, 122, 980, 606, 844, 65]], [[737, 50, 993, 198, 841, 818, 924, 187]]]


ct2 : [[[178, 607, 545, 241, 874, 324, 993, 300]], [[644, 836, 1001, 63, 326, 458, 282, 145]]]



final result:  0


In [38]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

6      0


AssertionError: 